In [3]:
import altair as alt
import geopandas as gpd
from shapely.ops import orient
# Define which attributes to lookup from airports.csv
cabi_stations = "https://raw.githubusercontent.com/mlinds/cabi-data/main/data/processed/stationLookup.csv"
connections_path = "https://raw.githubusercontent.com/mlinds/cabi-data/main/data/processed/connections_csv.csv"
wards_path = 'https://opendata.arcgis.com/datasets/a4442c906559456eb6ef3ea0898fe994_32.geojson'

dc_wards = gpd.read_file(wards_path)
dc_wards.geometry = dc_wards.geometry.apply(orient, args=(-1,))

basechart = (
    alt.Chart(dc_wards)
    .mark_geoshape(fill="None", stroke="gray")
    .properties(
        width=750 * 1.75,
        height=500 * 1.75,
    )
)
lookup_data = alt.LookupData(
    cabi_stations, key="short_name", fields=["name", "lat", "lon"]
)

## TODO add popularity to the stations data
# lookup_pop = alt.LookupData(connections_path, key="st", fields=["popularity"])

# Create mouseover selection
select_station_st = alt.selection_single(
    on="mouseover", nearest=True, fields=["st"], empty="none"
)

# Create mouseover selection
select_station_en = alt.selection_single(
    on="mouseover", nearest=True, fields=["en"], empty="none"
)

color_size_scale = "symlog"
colorscale = alt.Color(
    "popularity:Q", scale=alt.Scale(scheme="inferno", type=color_size_scale)
)

connections_forward = (
    alt.Chart(connections_path)
    .mark_rule()
    .encode(
        latitude="lat:Q",
        longitude="lon:Q",
        latitude2="lat2:Q",
        longitude2="lon2:Q",
        color=colorscale,
        strokeWidth=alt.StrokeWidth(
            "popularity:Q", scale=alt.Scale(type=color_size_scale)
        ),
        order=alt.Order("popularity:Q", sort="ascending"),
        # opacity = alt.Opacity('popularity:Q',scale=alt.Scale(type="linear"))
    )
    .transform_lookup(lookup="st", from_=lookup_data)
    .transform_lookup(lookup="en", from_=lookup_data, as_=["name2", "lat2", "lon2"]) 
    .transform_filter(select_station_st)
)
connections_backward = (
    alt.Chart(connections_path)
    .mark_rule()
    .encode(
        latitude="lat:Q",
        longitude="lon:Q",
        latitude2="lat2:Q",
        longitude2="lon2:Q",
        color=colorscale,
        strokeWidth=alt.StrokeWidth(
            "popularity:Q", scale=alt.Scale(type=color_size_scale)
        ),
        order=alt.Order("popularity:Q", sort="ascending"),
        # opacity = alt.Opacity('popularity:Q',scale=alt.Scale(type="linear"))
    )
    .transform_lookup(lookup="st", from_=lookup_data)
    .transform_lookup(lookup="en", from_=lookup_data, as_=["name2", "lat2", "lon2"]) 
    .transform_filter(select_station_en)
)


points = (
    alt.Chart(cabi_stations)
    .mark_circle(opacity=1)
    .encode(
        latitude="lat:Q",
        longitude="lon:Q",
        tooltip=["name:N"],
    )
    .transform_lookup(
        lookup="short_name",
        from_=alt.LookupData(cabi_stations, key="short_name", fields=["short_name"]),
        as_=["st"],
    )
    .transform_lookup(
        lookup="short_name",
        from_=alt.LookupData(cabi_stations, key="short_name", fields=["short_name"]),
        as_=["en"],
    )
    .add_selection(select_station_st)
    .add_selection(select_station_en)
)

combined_chart =  connections_forward + connections_backward + points
combined_chart.configure_view(stroke=None)


alt.LayerChart(...)

In [4]:
html_out = combined_chart.to_html()
with open("../reports/MapOutput.html", "w") as file:
    file.write(html_out)

# #  uncomment if a json file is needed
# with open('vega-lite-map.json','w') as file:
#     file.write(combined_chart.to_json())
